## License Plate Detection using Fuzzy Join 

### Start EVA server

We are reusing the start server notebook for launching the EVA server.

In [16]:
# !wget -nc "https://raw.githubusercontent.com/georgia-tech-db/eva/master/tutorials/00-start-eva-server.ipynb"
%run 00-start-eva-server.ipynb
cursor = connect_to_server()

nohup eva_server >> eva.log 2>&1 &
('03-17-2023 00:23:03 INFO  [catalog_manager:catalog_manager.py:_bootstrap_catalog:0076] Bootstrapping catalog\n', '03-17-2023 00:23:03 INFO  [timer:timer.py:log_elapsed_time:0045] Query Compile Time: 0.4414 sec\n', '03-17-2023 00:23:03 WARNING[create_udf_executor:create_udf_executor.py:exec:0042] UDF FastRCNNObjectDetector already exists, nothing added.\n')
('03-17-2023 00:23:03 INFO  [timer:timer.py:log_elapsed_time:0045] Query Compile Time: 0.0013 sec\n', '03-17-2023 00:23:03 WARNING[create_udf_executor:create_udf_executor.py:exec:0042] UDF Array_Count already exists, nothing added.\n', '03-17-2023 00:23:03 INFO  [timer:timer.py:log_elapsed_time:0045] Query Compile Time: 0.0007 sec\n')
('03-17-2023 00:23:03 WARNING[create_udf_executor:create_udf_executor.py:exec:0042] UDF Timestamp already exists, nothing added.\n', '03-17-2023 00:23:03 INFO  [timer:timer.py:log_elapsed_time:0045] Query Compile Time: 0.0009 sec\n', '03-17-2023 00:23:03 WARNING[crea

### Adding the images to EVADB for analysis

In [17]:
cursor.execute('DROP TABLE IF EXISTS MyImages;')
response = cursor.fetch_all()
print(response)

cursor.execute('LOAD IMAGE "workflows/images/Cars0.png" INTO MyImages;')
response = cursor.fetch_all()
print(response)

cursor.execute('LOAD IMAGE "workflows/images/Cars0.png" INTO MyImages;')
response = cursor.fetch_all()
print(response)

cursor.execute('LOAD IMAGE "workflows/images/Cars1.png" INTO MyImages;')
response = cursor.fetch_all()
print(response)

cursor.execute('LOAD IMAGE "workflows/images/Cars2.png" INTO MyImages;')
response = cursor.fetch_all()
print(response)

cursor.execute('LOAD IMAGE "workflows/images/Cars3.png" INTO MyImages;')
response = cursor.fetch_all()
print(response)

cursor.execute('LOAD IMAGE "workflows/images/Cars4.png" INTO MyImages;')
response = cursor.fetch_all()
print(response)

cursor.execute('LOAD IMAGE "workflows/images/Cars5.png" INTO MyImages;')
response = cursor.fetch_all()
print(response)

cursor.execute('LOAD IMAGE "workflows/images/Cars6.png" INTO MyImages;')
response = cursor.fetch_all()
print(response)

@status: ResponseStatus.SUCCESS
@batch: 
 Empty DataFrame
Columns: []
Index: []
@query_time: 0.02320384606719017
@status: ResponseStatus.SUCCESS
@batch: 
                            0
0  Number of loaded IMAGE: 1
@query_time: 0.07838605158030987
@status: ResponseStatus.FAIL
@batch: 
 None
@error: Load IMAGE failed: encountered unexpected error Duplicate File: workflows/images/Cars0.png already exists in the table MyImages
@status: ResponseStatus.SUCCESS
@batch: 
                            0
0  Number of loaded IMAGE: 1
@query_time: 0.02778487466275692
@status: ResponseStatus.SUCCESS
@batch: 
                            0
0  Number of loaded IMAGE: 1
@query_time: 0.03496135398745537
@status: ResponseStatus.SUCCESS
@batch: 
                            0
0  Number of loaded IMAGE: 1
@query_time: 0.025438737124204636
@status: ResponseStatus.SUCCESS
@batch: 
                            0
0  Number of loaded IMAGE: 1
@query_time: 0.02032211795449257
@status: ResponseStatus.SUCCESS
@batch: 


### License Plate Recognition

In [40]:
cursor.execute("""CREATE UDF IF NOT EXISTS OCRExtractor
                  INPUT  (frame NDARRAY UINT8(3, ANYDIM, ANYDIM))
                  OUTPUT (labels NDARRAY STR(10),
                          bboxes NDARRAY FLOAT32(ANYDIM, 4),
                          scores NDARRAY FLOAT32(ANYDIM))
                  TYPE  OCRExtraction
                  IMPL  '../eva/udfs/ocr_extractor.py' ;
      """)
response = cursor.fetch_all()
print(response)

cursor.execute("""CREATE UDF IF NOT EXISTS FuzzDistance
                    INPUT (Input_Array1 NDARRAY ANYTYPE, Input_Array2 NDARRAY ANYTYPE)
                    OUTPUT (distance FLOAT(32, 7))
                    TYPE NdarrayUDF
                    IMPL "../eva/udfs/ndarray/fuzzy_join.py";
                    """)
response = cursor.fetch_all()
print(response)

@status: ResponseStatus.SUCCESS
@batch: 
                                                       0
0  UDF FuzzDistance successfully added to the database.
@query_time: 0.038568850606679916


In [31]:
cursor.execute("CREATE TABLE IF NOT EXISTS LicensePlateCSV(id INTEGER UNIQUE, label TEXT(30));")
response = cursor.fetch_all()
print(response)

@status: ResponseStatus.SUCCESS
@batch: 
 Empty DataFrame
Columns: []
Index: []
@query_time: 0.05013679899275303


In [33]:
cursor.execute("LOAD CSV 'data.csv' INTO LicensePlateCSV;")
response = cursor.fetch_all()
print(response)


@status: ResponseStatus.SUCCESS
@batch: 
         CSV  Number of loaded frames
0  data.csv                        5
@query_time: 0.024829771369695663


In [27]:
cursor.execute('SELECT OCRExtractor(data) FROM MyImages;')
res = cursor.fetch_all()
print(res)

@status: ResponseStatus.SUCCESS
@batch: 
             ocrextractor.labels  \
0                  [KLL1CA2555]   
1          [Coi, rrm, PGoMN112]   
2                    [PRErINLB]   
3                    [DZIZ YXR]   
4                  [pPuie, BES]   
5      [Wet, alamy stock photo]   
6  [802.LIN, Fay, VIRGINIA, 07]   

                                                                                   ocrextractor.bboxes  \
0                                                   [[[241, 117], [424, 117], [424, 178], [241, 178]]]   
1  [[[320, 96], [342, 96], [342, 104], [320, 104]], [[56, 100], [90, 100], [90, 108], [56, 108]], [...   
2                                                   [[[227, 173], [269, 173], [269, 193], [227, 193]]]   
3                                                   [[[150, 128], [250, 128], [250, 158], [150, 158]]]   
4  [[[175.83355283634924, 85.05243648347647], [388.302989236221, 113.80042194340535], [367.16644716...   
5   [[[362, 274], [384, 274], [384, 280],

In [ ]:
cursor.execute('SELECT OCRExtractor(data) FROM MyImages;')
res = cursor.fetch_all()
print(res)

In [1]:
cursor.execute('SELECT * FROM MyImages JOIN LATERAL OCRExtractor(data) AS T(a,b,c) JOIN LicensePlateCSV B ON FuzzDistance(T.a, B.label) > 50;')
res = cursor.fetch_all()
print(res)


NameError: name 'cursor' is not defined